In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import black


def print_res(old_d: dict, result, just_ans=True):
    d = old_d.copy()
    d["tools"] = json.loads(d["tools"])
    if just_ans:
        del d["tools"]
    d["answers"] = eval(d["answers"])
    print("-" * 12, end="\n\n\n")
    print(black.format_str(str(d), mode=black.Mode()))
    print("-" * 12, end="\n\n\n")
    print(result)

In [3]:
import re
from parser import parse


def handle_response(response):
    pattern = re.compile(r"(?s)```shonqol\s*([\s\S]*?)\s*```")
    matches = pattern.findall(response)

    # for code_block in matches:
    return parse(matches[0])

In [4]:
def calc_hit_fn_arg(answers, extracted):
    hit_fn = 0
    hit_arg = 0
    tot_args = 0

    # Filter out non-dictionary entries and entries missing keys
    valid_answers = [
        a for a in answers if isinstance(a, dict) and "name" in a and "arguments" in a
    ]
    valid_extracted = [
        e for e in extracted if isinstance(e, dict) and "name" in e and "arguments" in e
    ]

    # Create a lookup for extracted functions by name
    extracted_lookup = [(e["name"], e) for e in valid_extracted]
    fn_names_set= set([e["name"] for e in valid_extracted])
    # Create a lookup for extracted functions by name mapping to a list of their 'arguments' lists
    extracted_lookup = {}
    for e in valid_extracted:
        extracted_lookup.setdefault(e["name"], []).append(e["arguments"])

    for a in valid_answers:
        fn_name = a["name"]
        if fn_name in extracted_lookup:
            hit_fn += 1
            ans_args = list(a["arguments"].values())
            tot_args += len(ans_args)
            for aarg in ans_args:
                # Check across all extracted function calls for this name, order doesn't matter
                if any(aarg in ext_args for ext_args in extracted_lookup[fn_name]):
                    hit_arg += 1

    print("hit_fn", hit_fn, "out of", len(valid_answers))
    print("hit_arg", hit_arg, "out of", tot_args)
    return (hit_fn,len(valid_answers)), (hit_arg, tot_args)


# Example data:
extracted = [
    {"name": "get_id", "arguments": ["Pulp Fiction"]},
    {"name": "search_torrents", "arguments": ["Pulp Fiction torrents"]},
    "Pulp Fiction torrents",
]


answers = [
    {"name": "get_id", "arguments": {"q": "Pulp Fiction"}},
    {
        "name": "search_torrents",
        "arguments": {"keywords": "Pulp Fiction", "quantity": 10},
    },
]

calc_hit_fn_arg(answers, extracted)

hit_fn 2 out of 2
hit_arg 1 out of 3


((2, 2), (1, 3))

In [5]:
import pandas as pd

# df = pd.read_json("hf://datasets/Salesforce/xlam-function-calling-60k/xlam_function_calling_60k.json")
df = pd.read_json("./xlam_function_calling_60k.json")

In [6]:
from dotenv import load_dotenv

load_dotenv()

True

In [8]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers.string import StrOutputParser
from prompts import main_prompt

prompt = ChatPromptTemplate.from_messages(
    [("system", main_prompt), ("user", "{query}")]
)
llm = prompt | ChatOpenAI(model="gpt-4o-mini", temperature=0) | StrOutputParser()

In [10]:
d = df.iloc[6].to_dict()
result = llm.invoke(d)
print_res(d, result)

------------


{
    "id": 6,
    "query": "Identify the ID of 'Pulp Fiction' and search for torrents related to it.",
    "answers": [
        {"name": "get_id", "arguments": {"q": "Pulp Fiction"}},
        {
            "name": "search_torrents",
            "arguments": {"keywords": "Pulp Fiction", "quantity": 10},
        },
    ],
}

------------


```shonqol
movie_id = get_id("Pulp Fiction");
search_torrents(concat("Pulp Fiction", " torrent"), 40);
```


In [11]:
import json


extracted_functions = handle_response(result)
print("*" * 20)
print((extracted_functions))
print("*" * 20)
print(d["answers"])
print("*" * 20)
calc_hit_fn_arg(json.loads( d["answers"]), extracted_functions)

Extracted function calls from AST:
[
    {
        "name": "get_id",
        "arguments": [
            "Pulp Fiction"
        ]
    },
    {
        "name": "search_torrents",
        "arguments": [
            "Pulp Fiction torrent",
            40
        ]
    },
    "Pulp Fiction torrent"
]
********************
[{'name': 'get_id', 'arguments': ['Pulp Fiction']}, {'name': 'search_torrents', 'arguments': ['Pulp Fiction torrent', 40]}, 'Pulp Fiction torrent']
********************
[{"name": "get_id", "arguments": {"q": "Pulp Fiction"}}, {"name": "search_torrents", "arguments": {"keywords": "Pulp Fiction", "quantity": 10}}]
********************
hit_fn 2 out of 2
hit_arg 1 out of 3


line 2:55 extraneous input ';' expecting {<EOF>, Identifier}


(2, 1)

In [41]:
d = df.iloc[9].to_dict()
result = llm.invoke(d)
print_res(d, result)

------------


{
    "id": 9,
    "query": "Is 'https://www.apple.com' available in the Wayback Machine on September 9, 2015?",
    "answers": [
        {
            "name": "availability",
            "arguments": {"url": "https://www.apple.com", "timestamp": "20150909"},
        }
    ],
}

------------


```shonqol
availability("https://www.apple.com", "20150909");
```


In [43]:
extracted_functions = handle_response(result)
calc_hit_fn_arg(json.loads( d["answers"]), extracted_functions)

Extracted function calls from AST:
[
    {
        "name": "availability",
        "arguments": [
            "https://www.apple.com",
            "20150909"
        ]
    }
]
hit_fn 1 out of 1
hit_arg 2 out of 2


line 1:49 extraneous input ';' expecting {<EOF>, Identifier}


(1, 2)

In [44]:
d = df.iloc[103].to_dict()
result = llm.invoke(d)
print_res(d, result, just_ans=False)

------------


{
    "id": 103,
    "query": "What is the recommended daily calorie intake and macronutrient distribution for a 30-year-old male, weighing 75 kg, 180 cm tall, moderately active, aiming to maintain weight?",
    "answers": [
        {
            "name": "calculate_calorie_intake",
            "arguments": {
                "weight_kg": 75,
                "height_cm": 180,
                "age": 30,
                "sex": "male",
                "activity_level": 3,
                "goal": "maintain",
            },
        }
    ],
    "tools": [
        {
            "name": "calculate_calorie_intake",
            "description": "Calculates the recommended daily calorie intake and macronutrient distribution based on personal characteristics and goals.",
            "parameters": {
                "weight_kg": {
                    "description": "Body weight in kilograms.",
                    "type": "float",
                },
                "height_cm": {"descript

In [45]:
extracted_functions = handle_response(result)
calc_hit_fn_arg(json.loads( d["answers"]), extracted_functions)

Extracted function calls from AST:
[
    {
        "name": "calculate_calorie_intake",
        "arguments": [
            75,
            180,
            30,
            "male",
            3,
            "maintain"
        ]
    }
]
hit_fn 1 out of 1
hit_arg 6 out of 6


line 1:60 extraneous input ';' expecting {<EOF>, Identifier}


(1, 6)

In [46]:
d = df.iloc[586].to_dict()
result = llm.invoke(d)
print_res(d, result, just_ans=True)

------------


{
    "id": 586,
    "query": "I'm curious about the neuronal activity rate when the synaptic input rate is 100 per second, with a synaptic weight of 0.5 and a decay constant of 0.05. Also, could you check if 17 is a prime number and solve the quadratic equation 2x^2 - 3x + 1 = 0?",
    "answers": [
        {
            "name": "neuronal_activity_rate",
            "arguments": {
                "synaptic_input_rate": 100,
                "synaptic_weight": 0.5,
                "decay_constant": 0.05,
            },
        },
        {"name": "is_prime", "arguments": {"num": 17}},
        {"name": "solve_quadratic", "arguments": {"a": 2, "b": -3, "c": 1}},
    ],
}

------------


```shonqol
neuronal_activity = neuronal_activity_rate(100, 0.5, 0.05);
is_seventeen_prime = is_prime(17);
quadratic_roots = solve_quadratic(2, -3, 1);
```


In [50]:
extracted_functions = handle_response(result)
calc_hit_fn_arg(json.loads( d["answers"]), extracted_functions)

Extracted function calls from AST:
[
    {
        "name": "neuronal_activity_rate",
        "arguments": [
            100,
            0.5,
            0.05
        ]
    },
    {
        "name": "is_prime",
        "arguments": [
            17
        ]
    },
    {
        "name": "solve_quadratic",
        "arguments": [
            2,
            -3,
            1
        ]
    }
]
hit_fn 3 out of 3
hit_arg 7 out of 7


(3, 7)

In [12]:
d = df.iloc[998].to_dict()
result = llm.invoke(d)
print_res(
    d,
    result,
)

------------


{
    "id": 998,
    "query": "List all tokens on the Ethereum network.",
    "answers": [{"name": "tokens", "arguments": {"network": "ethereum"}}],
}

------------


```shonqol
tokens("ethereum");
```


In [13]:
extracted_functions = handle_response(result)
calc_hit_fn_arg(json.loads( d["answers"]), extracted_functions)

Extracted function calls from AST:
[
    {
        "name": "tokens",
        "arguments": [
            "ethereum"
        ]
    }
]
hit_fn 1 out of 1
hit_arg 1 out of 1


line 1:18 extraneous input ';' expecting {<EOF>, Identifier}


(1, 1)

In [17]:
import json

for _, row in df.iterrows():

    v = json.loads(str(row["answers"]))
    if not isinstance(v, list):
        continue
    if len(v) > 3:
        d = row.to_dict()
        
        break
result = llm.invoke(d)
print_res(d, result, just_ans=True)

------------


{
    "id": 107,
    "query": "I'm conducting a research study and need the men's marks for points values of 10, 20, 30, 40, and 50 to observe the trend in the lower range.",
    "answers": [
        {"name": "marks_men_points", "arguments": {"points": 10}},
        {"name": "marks_men_points", "arguments": {"points": 20}},
        {"name": "marks_men_points", "arguments": {"points": 30}},
        {"name": "marks_men_points", "arguments": {"points": 40}},
        {"name": "marks_men_points", "arguments": {"points": 50}},
    ],
}

------------


```shonqol
marks_10 = marks_men_points(10);
marks_20 = marks_men_points(20);
marks_30 = marks_men_points(30);
marks_40 = marks_men_points(40);
marks_50 = marks_men_points(50);
```


In [39]:
extracted_functions = handle_response(result)
calc_hit_fn_arg(json.loads( d["answers"]), extracted_functions)

hit_fn 3 out of 3
hit_arg 5 out of 5


((3, 3), (5, 5))

In [21]:
df.sample(1)

,id,query,answers,tools
18675,18675,Fetch the 5-day forecast for London's coordina...,"[{""name"": ""daily_forecast_5_days"", ""arguments""...","[{""name"": ""alerts_active_zone_zoneid"", ""descri..."


In [10]:
# Choose 10 random entries from the DataFrame
import os


sample_df = df.sample(10)
if not os.path.exists("outputs"):
    os.makedirs("outputs")
for idx, row in sample_df.iterrows():
    d = row.to_dict()
    # d=df.iloc[19881].to_dict()

    # Call the language model as done in previous cells
    result = llm.invoke(d)
    print_res(d, result)
    
    # Process extracted functions and calculate hit metrics
    extracted_functions = handle_response(result)
    hit_result=calc_hit_fn_arg(json.loads(d["answers"]), extracted_functions)
    print("-" * 40)
    d['tools']=json.loads(d["tools"])
    d['answers']=json.loads(d["answers"])
    # Write d, result, and extracted_functions to a json file
    data_to_save = {
        'id':d['id'],
        "d": d,
        "result": result,
        "extracted_functions": extracted_functions,
        "hit_results":{ "hit_fn":hit_result[0], "hit_arg":hit_result[1]}
    }
    with open(f"outputs/output_{d['id']}.json", "w") as f:
        json.dump(data_to_save, f, indent=2)

------------


{
    "id": 4113,
    "query": "What's the neuronal activity rate if there are 50 synaptic inputs per second with a weight of 0.8 and a decay of 0.05?",
    "answers": [
        {
            "name": "neuronal_activity_rate",
            "arguments": {
                "synaptic_input_rate": 50,
                "synaptic_weight": 0.8,
                "decay_constant": 0.05,
            },
        }
    ],
}

------------


```shonqol
neuronal_activity_rate(50, 0.8, 0.05);
```
hit_fn 1 out of 1
hit_arg 3 out of 3
----------------------------------------


line 1:37 extraneous input ';' expecting {<EOF>, Identifier}


------------


{
    "id": 57378,
    "query": "What is the electric field intensity produced by a charge of 10 coulombs at a distance of 10 meters?",
    "answers": [
        {
            "name": "calculate_electric_field",
            "arguments": {"charge": 10, "distance": 10},
        }
    ],
}

------------


```shonqol
calculate_electric_field(charge=10, distance=10);
```
hit_fn 1 out of 1
hit_arg 2 out of 2
----------------------------------------


line 1:48 extraneous input ';' expecting {<EOF>, Identifier}


------------


{
    "id": 21588,
    "query": "I'm interested in seeing the first 10 and then the next 10 potentially undervalued large cap stocks. Can you provide that information?",
    "answers": [
        {"name": "market_undervalued_large_caps", "arguments": {"start": 0}},
        {"name": "market_undervalued_large_caps", "arguments": {"start": 10}},
    ],
}

------------


```shonqol
firstBatch = market_undervalued_large_caps(0);
secondBatch = market_undervalued_large_caps(10);
print(firstBatch);
print(secondBatch);
```
hit_fn 2 out of 2
hit_arg 2 out of 2
----------------------------------------


...

------------


{
    "id": 52360,
    "query": "Can Gyarados learn the move 'Hydro Pump'?",
    "answers": [
        {
            "name": "get_pokemon_move_info",
            "arguments": {"pokemon_name": "gyarados", "move_name": "hydro pump"},
        }
    ],
}

------------


```shonqol
get_pokemon_move_info("gyarados", "hydro pump");
```
hit_fn 1 out of 1
hit_arg 2 out of 2
----------------------------------------


line 1:47 extraneous input ';' expecting {<EOF>, Identifier}


------------


{
    "id": 1524,
    "query": "I need to see the QR code for '7539182640' and also learn how to redeploy its Docker instance.",
    "answers": [
        {"name": "getqrcode", "arguments": {"phone_id": "7539182640"}},
        {"name": "redeploy", "arguments": {"phone_id": "7539182640"}},
    ],
}

------------


```shonqol
getqrcode("7539182640");
redeploy("7539182640");
```
hit_fn 2 out of 2
hit_arg 2 out of 2
----------------------------------------


line 1:23 extraneous input ';' expecting {<EOF>, Identifier}
line 2:22 extraneous input ';' expecting {<EOF>, Identifier}


------------


{
    "id": 59937,
    "query": "Search for location information for Mexico City, Mexico and Seoul, South Korea in XML format.",
    "answers": [
        {"name": "search_api", "arguments": {"q": "Mexico City", "format": "xml"}},
        {"name": "search_api", "arguments": {"q": "Seoul", "format": "xml"}},
    ],
}

------------


```shonqol
mexicoCityInfo = search_api("Mexico City, Mexico", "xml");
seoulInfo = search_api("Seoul, South Korea", "xml");
```
hit_fn 2 out of 2
hit_arg 2 out of 4
----------------------------------------


In [44]:
handle_response("""```shonqol
merged_list = merge_sorted_lists([10, 20, 30], [15, 25, 35]);
standard_deviation = calculate_standard_deviation(merged_list);
alimony = california_alimony(3500, 1500, 3);
```""")

[{'name': 'merge_sorted_lists', 'arguments': ['[10,20,30]', '[15,25,35]']},
 {'name': 'calculate_standard_deviation',
  'arguments': [{'type': 'Identifier', 'name': 'merged_list'}]},
 {'name': 'california_alimony', 'arguments': [3500, 1500, 3]}]

In [ ]:
# d['tools']=json.loads(d["tools"])
# d['answers']=json.loads(d["answers"])
print(json.dumps(d,indent=4))

{
    "id": 6536,
    "query": "Fetch the video count and the published date of the channel 'UC-M9jKQyJqJh2Dz6eIq9Htw'. Also, retrieve the details about the channel's branding settings.",
    "answers": [
        {
            "name": "channel_details",
            "arguments": {
                "is_id": "UC-M9jKQyJqJh2Dz6eIq9Htw",
                "part": "statistics,snippet"
            }
        },
        {
            "name": "channel_details",
            "arguments": {
                "is_id": "UC-M9jKQyJqJh2Dz6eIq9Htw",
                "part": "brandingSettings"
            }
        }
    ],
    "tools": [
        {
            "name": "channel_details",
            "description": "Fetches details of a YouTube channel given its ID and desired parts using the RapidAPI service.",
            "parameters": {
                "is_id": {
                    "description": "The ID of the YouTube channel to fetch details for.",
                    "type": "str",
                    "de

In [12]:
print(d)

{'id': 59937, 'query': 'Search for location information for Mexico City, Mexico and Seoul, South Korea in XML format.', 'answers': [{'name': 'search_api', 'arguments': {'q': 'Mexico City', 'format': 'xml'}}, {'name': 'search_api', 'arguments': {'q': 'Seoul', 'format': 'xml'}}], 'tools': [{'name': 'find_places', 'description': 'Searches for places by name to obtain a place_id for use with Weather Forecast Endpoints and retrieves detailed geographical information (e.g., country, region, elevation, timezone) for a given location.', 'parameters': {'text': {'description': 'Name of the place to search.', 'type': 'str', 'default': 'fishermans wharf'}, 'language': {'description': 'Language in which the place names will be returned. Available options include:', 'type': 'str, optional', 'default': 'en'}}}, {'name': 'search_autocomplete_api', 'description': 'Search or Autocomplete API returns matching cities and towns based on the query.', 'parameters': {'q': {'description': 'Query parameter used